In [8]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np    

In [9]:
#clean meteorological data time stamps and eliminate seemingly irrelevant columns
meteorological_df = pd.read_excel('hw03-data/Sensor Data/Meteorological Data.xlsx')
if 'Date' in meteorological_df.columns:
    meteorological_df['timeStamp'] = pd.to_datetime(meteorological_df['Date'], errors='coerce')
else:
    print('Column "Date" not found')
meteorological_df = meteorological_df.drop(columns=['Date'])
meteorological_df = meteorological_df.drop(columns=['Unnamed: 3'])
meteorological_df = meteorological_df.drop(columns=['Elevation (m)'])

In [10]:
#clean sensor data time stamps 
sensor_df = pd.read_excel('hw03-data/Sensor Data/Sensor Data.xlsx')
if 'Date Time ' in sensor_df.columns:
    sensor_df['timeStamp'] = pd.to_datetime(sensor_df['Date Time '], errors='coerce')
else:
    print('Column "Date Time" not found')
sensor_df = sensor_df.drop(columns=['Date Time '])

In [11]:
if 'Wind Direction' in meteorological_df.columns:
    meteorological_df['direction'] = (meteorological_df['Wind Direction'] / 45).round()

meteorological_df = meteorological_df[~meteorological_df['direction'].isna()]

#meteorological_df['direction'] = meteorological_df['direction'].replace({9.0: 'B'})
meteorological_df['direction'] = meteorological_df['direction'].replace({1.0: 'N'})
meteorological_df['direction'] = meteorological_df['direction'].replace({2.0: 'NE'})
meteorological_df['direction'] = meteorological_df['direction'].replace({3.0: 'E'})
meteorological_df['direction'] = meteorological_df['direction'].replace({4.0: 'SE'})    
meteorological_df['direction'] = meteorological_df['direction'].replace({5.0: 'S'})
meteorological_df['direction'] = meteorological_df['direction'].replace({6.0: 'SW'})   
meteorological_df['direction'] = meteorological_df['direction'].replace({7.0: 'W'})
meteorological_df['direction'] = meteorological_df['direction'].replace({8.0: 'NW'})

#count_sw = (meteorological_df['direction'] == 'SW').sum()
#print("Count SW: ", count_sw)
sw_df = meteorological_df[meteorological_df['direction'] == 'SW']
sw_df.head()

,Wind Direction,Wind Speed (m/s),timeStamp,direction
7,262.1,2.9,2016-04-01 21:00:00,SW
8,262.3,2.9,2016-04-02 00:00:00,SW
9,271.8,2.8,2016-04-02 03:00:00,SW
10,279.8,1.5,2016-04-02 06:00:00,SW
11,272.3,1.3,2016-04-02 09:00:00,SW


In [12]:
# Filter sensor_df for rows with dates in unique_sw_dates
unique_sw_dates = sw_df['timeStamp'].dt.date.unique()  # Make sure 'timeStamp' is in sw_df
filtered_sensor_df = sensor_df[sensor_df['timeStamp'].isin(unique_sw_dates)]
filtered_sensor_df = filtered_sensor_df[sensor_df['Monitor']==9]
filtered_sensor_df.head()

/var/folders/lc/3lf1vl6d5278xt54g66z2f880000gn/T/ipykernel_31263/3693577503.py:3: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  filtered_sensor_df = sensor_df[sensor_df['timeStamp'].isin(unique_sw_dates)]
/var/folders/lc/3lf1vl6d5278xt54g66z2f880000gn/T/ipykernel_31263/3693577503.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_sensor_df = filtered_sensor_df[sensor_df['Monitor']==9]


,Chemical,Monitor,Reading,timeStamp
15,Appluimonia,9,0.272886,2016-04-01
21,Chlorodinine,9,0.197804,2016-04-01
26,Methylosmolene,9,0.144993,2016-04-01
29,AGOC-3A,9,0.105787,2016-04-01
1711,Methylosmolene,9,0.306977,2016-04-03


In [16]:
# Create a DataFrame
filtered_sensor_df = pd.DataFrame(filtered_sensor_df)

# Calculate the average reading for each chemical
average_readings = filtered_sensor_df.groupby('Chemical', as_index=False)['Reading'].mean()

# Create a bar chart with average readings
fig = px.bar(
    average_readings,
    x='Chemical',
    y='Reading',
    title='Average Chemical Readings for Sensor 9 on Days with SW Winds',
    labels={'Reading': 'Average Reading Value', 'Chemical': 'Chemical Name'}
)

fig.update_layout(
    yaxis=dict(range=[0, 1], title='Chemical Reading'),
    xaxis=dict(title='Chemical'),
)


# Show the plot
fig.show()
